In [2]:
cd ..

/content/gdrive/MyDrive/Github/lc-classification


In [8]:
import pandas as pd
import numpy as np
from src.embeddings.longformer import Book2Vec

In [ ]:
# import dataset
df = pd.read_json('./data/dataset_B.json')
metadata = pd.read_json('./data/metadata.json')
X_embeddings = Book2Vec.load_embeddings('./work/embeddings_B_last4layers.pkl')

from 